# Packaging tests

In [1]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument #
import json #
import multiprocessing#

import joblib 
import pandas as pd
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import joblib
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from pprint import pprint
import math




In [3]:
df = joblib.load('../../../raw_data/processed_data.joblib')
df['tag_language'] = df['tag_language'].fillna(value='en')

In [4]:
# select english jobs
df_eng = df.copy()
df_eng = df_eng[df_eng['tag_language'] == 'en']
df_eng.reset_index(inplace=True)
df_eng.drop(columns='index', inplace=True)

In [5]:
# join strings
def join_strings(text):
    return ' '.join(text)

# lemmatize
def lemmatize_words(word):
    lemmatizer = WordNetLemmatizer()
    lemmatized = lemmatizer.lemmatize(word)

    return lemmatized

# remove stopwords
def remove_stopwords(text):

    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text) 
    text = [w for w in word_tokens if not w in stop_words] 
  
    return text

In [6]:
# process text
df_eng['clean'] = df_eng['job_text_tokenized'].apply(join_strings).apply(lemmatize_words)\
    .apply(remove_stopwords)


# functions for pipeline

In [11]:
def tag_texts(texts):
    ''' input: pandas series of datasets to use in the model
        returns texts with identification tag
    '''
    texts_tagged = [TaggedDocument(text, tags=['tag_'+str(tag)]) for tag, text in enumerate(texts)]

    return texts_tagged

texts_tagged = tag_texts(texts)
#texts_tagged[0]

In [24]:
texts_tagged_small = texts_tagged[:3000]

In [26]:
def instanciate_Doc2Vec(text_tagged):
    '''instanciates model, using dbow (d=0)'''
    cores = multiprocessing.cpu_count()
    model_dbow = Doc2Vec(documents=text_tagged,
                         dm=0,
                         alpha=0.025,
                         vector_size=len(df), 
                         min_count=1,
                         workers=cores,
                        )

In [27]:
def train_Doc2Vec(texts_tagged_small):
    ''' trains model'''
    model_dbow.train(text_tagged,
                     total_examples=model_dbow.corpus_count, 
                     epochs=15,
                    )

In [ ]:
def save_Doc2Vec(path):
    '''saves trained Doc2Vec
    input: path to location 
    '''
    model_dbow.save(path)

In [ ]:
# instanciate_Doc2Vec(texts_tagged_small)
# train_Doc2Vec(text_tagged_small)

In [ ]:
data_to_train = texts_tagged_small # texts_tagged_small, texts_tagged

# build vocabulary with CBOW (dm=0)
cores = multiprocessing.cpu_count()
model_dbow = Doc2Vec(documents=data_to_train,
                     dm=0,
                     alpha=0.025,
                     vector_size=len(data_to_train), 
                     min_count=1,
                     workers=cores)

# train the model
model_dbow.train(data_to_train, total_examples=model_dbow.corpus_count, epochs=15)

In [40]:
# infer vector and get similarity
def get_similar_jobs(tokenized_job, number_offers):
    ''' input: tokenized job offers, number of offers 
        returns tags of top x most similar job offers and similarity probabilities
    '''

    # infer vector from text 
    infer_vector = model_loaded.infer_vector(tokenized_job)
    # find similar offers
    similar_documents = model_loaded.docvecs.most_similar([infer_vector], topn = number_offers)

    return similar_documents


## start pipeline

In [37]:
class Doc2Vec_Pipeline:
    
    # instanciate class
    def __init__(self, path=None, texts=None):
        if path:
            self.d2v_model = Doc2Vec.load(path)
        else:
            cores = multiprocessing.cpu_count()
            self.d2v_model = Doc2Vec(documents=df,
                     dm=0,
                     alpha=0.025,
                     vector_size=len(df), 
                     min_count=1,
                     workers=cores)
    
    # build vocabulary
    def tag_docs(self, texts):
        ''' input: pandas series of datasets to use in the model
            returns texts with identification tag
        '''
        self.texts_tagged = [TaggedDocument(text, tags=['tag_'+str(tag)]) for tag, text in enumerate(texts)]

        return self.texts_tagged


    
    
    # train model
    
    # define most similar jobs
    
    # 